# Setup 

## Import packages 

In [1]:
# General 
import os 
import numpy as np
# For running bash scripts from inside python ... 
import subprocess
# For manipulating string objects 
import re
# for generating any necessary directories
import pathlib 

In [2]:
# For working with sequence objects 
from Bio.Seq import Seq

In [3]:
# For fetching sequences from Entrez 
from Bio import Entrez
from Bio import SeqIO

In [4]:
# For extracting features 
from Bio.SeqFeature import SeqFeature, FeatureLocation
# For creating SeqRecord objects 
from Bio.SeqRecord import SeqRecord

## Misc

In [5]:
Entrez.email = "kehaliwoldemichael@gmail.com"  # Always tell NCBI who you are

# Functions 

## Sequence

In [6]:
def seq_returnEntrez(sequenceID, retType):
    with Entrez.efetch(
        db="nucleotide", rettype=retType, retmode="text", id=sequenceID
    ) as handle:
        seqRecord = SeqIO.read(handle, "gb")  # using "gb" as an alias for "genbank"
        
    handle = Entrez.efetch(db="nucleotide", id=sequenceID, rettype=retType, retmode="text")
    
    return seqRecord, handle 

## Metrics 

In [7]:
# Returns GC content 
def metric_gcContent(sequence):
    return (sequence.count("G") + sequence.count("C"))/(len(sequence))

# Sequence

## Loading sequences 

In [8]:
geneName = 'Fezf2'
fileName = os.getcwd() + '/Output/biomaRt/Reverse_' + geneName + '.fasta'
fileName

'/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/biomaRt/Reverse_Fezf2.fasta'

In [9]:
# Loading sequences for gene exons 
exon_records = list(SeqIO.parse(fileName, "fasta"))
exon_records  

[SeqRecord(seq=Seq('TAGTGGTTCTGTTTATTGAGTCATATATGTGTAATATTCCGTGTTCGCTTGTAC...TCC'), id='Fezf2', name='Fezf2', description='Fezf2', dbxrefs=[]),
 SeqRecord(seq=Seq('CTTTTTCCCCCACCGCCAAGGAGATGCGTTCCGAGCCATGCAGCGTGTCTCTTC...CTA'), id='Fezf2', name='Fezf2', description='Fezf2', dbxrefs=[]),
 SeqRecord(seq=Seq('CTTGCCGCACACTTCGCAGGTGAAGTTTTTGGGTTTGCTGTCAGTAGAGCCCCC...AGT'), id='Fezf2', name='Fezf2', description='Fezf2', dbxrefs=[])]

In [10]:
len(exon_records[2].seq)

890

In [11]:
seq_record = exon_records[2]

In [12]:
metric_gcContent(seq_record.seq)

0.6348314606741573

## Selecting sensor 

In [13]:
seq_record

SeqRecord(seq=Seq('CTTGCCGCACACTTCGCAGGTGAAGTTTTTGGGTTTGCTGTCAGTAGAGCCCCC...AGT'), id='Fezf2', name='Fezf2', description='Fezf2', dbxrefs=[])

In [14]:
seq = seq_record.seq
len(seq)

890

In [15]:
# Checks if continuous open reading frame by translating to stop ... 
def check_cORF(sequence):
    return len(sequence.translate(to_stop=True)) == len(sequence)/3

In [16]:
def return_inFrame(sequence):
    # Generating list of codons in sequence 
    strSeq = str(sequence)
    codons = [strSeq for strSeq in re.split(r'(\w{3})', strSeq) if strSeq]
    
    # Number of in frame TGG and ATG 
    num_inF_TGG = codons.count('TGG')
    num_inF_ATG = codons.count('ATG')
    
    # Indices of TGG and ATG 
    indicesTGG = np.array([key for key, val in enumerate(codons) if val == 'TGG'])*3
    indicesATG = np.array([key for key, val in enumerate(codons) if val == 'ATG'])*3
    
    return num_inF_TGG, num_inF_ATG, indicesTGG, indicesATG

In [128]:
def generate_sesRNA(sequence, length):
    start = 0
    center = length/2
    
    global numPass, total, numTGG  
    numPass = 0 
    total = 0 
    numTGG = []
    
    sesSeq = []
    startSeq = []
    
    while(start <= (len(sequence) - length)):
        # Defining current sub-sequence to process 
        subsequence = sequence[start:(start+length)]
        
        # Genrating index of any stop codons 
        stopCodons = ['TAG', 'TAA', 'TGA']
        indiciesStop = []
        for codons in stopCodons:
            indiciesStop.extend([m.start() for m in re.finditer(codons, str(subsequence))])
        
        # GC content 
        gcContent = metric_gcContent(subsequence)*100
        # Index of last ATG and TGG 
        lastATG = 0 
        if(subsequence.count('ATG') != 0):
            lastATG = [m.start() for m in re.finditer('ATG', str(subsequence))][-1]
        lastTGG = [m.start() for m in re.finditer('TGG', str(subsequence))][-1]
        # Getting indicies of TGG 
        indiciesTGG = [m.start() for m in re.finditer('TGG', str(subsequence))]
        # Generating arrays indicies for TGGs and stop codons 
        arrayStop = np.array(indiciesStop)
        arrayIndicies = np.array(indiciesTGG) 
        centralTGGs = arrayIndicies[abs(arrayIndicies - center) < 10]
        
        num_inF_TGG, num_inF_ATG, indices_inF_TGG, indices_inF_ATG = return_inFrame(subsequence)
        numATG = subsequence.count('ATG')
        
        # Only proceed if passed 
        # cond1 = len(indiciesStop) < 4 
        cond1 = check_cORF(subsequence)
        cond2 = num_inF_TGG > 2
        cond3 = num_inF_ATG == 0 
        
        cond4 = gcContent > 40
        cond5 = gcContent < 65
        
        # cond4 = lastATG < lastTGG 
        # Checking if TGG near center of subsequence 
        cond6 = any(abs(x - center) < 10 for x in indices_inF_TGG)
        # Checking if any central array is more than 10 by away from stop  
        cond7 = any((min(abs(arrayStop - i)) >= 20) for i in centralTGGs)
        
        if(cond1 & cond2 & cond3 & cond4 & cond5 & cond6 & cond7):
            numPass += 1
            numTGG.append(subsequence.count('TGG'))
            sesSeq.append(subsequence)
            startSeq.append(start)
            
        total += 1 
        # Updating start index 
        start += 1 
    
    return sesSeq, startSeq

In [18]:
seq.count('ATG')

11

In [129]:
generate_sesRNA(seq, 204)

([Seq('CTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGG...GTT'),
  Seq('GTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTG...GAT'),
  Seq('CGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGC...GAC')],
 [177, 180, 183])

In [20]:
len(sesRNAs)

3

In [132]:
def generate_sesRNAs_multiExon(exon_records):
    tempAll_sesRNAs = []
    tempAll_startSeq = []
    
    for record in exon_records:
        tempSeq = record.seq 
        temp_sesRNAs, temp_startSeq = generate_sesRNA(tempSeq, 204)
        tempAll_sesRNAs.extend(temp_sesRNAs)
        tempAll_startSeq.extend(temp_startSeq)
        
        # Printing number of passed sequences for current exon 
        print(len(temp_sesRNAs))
    return tempAll_sesRNAs, tempAll_startSeq

In [133]:
multiExon_sesRNAs, multi_startSeq = generate_sesRNAs_multiExon(exon_records)

0
0
3


In [134]:
multi_startSeq

[177, 180, 183]

In [120]:
for sequence in multiExon_sesRNAs:
    num_inF_TGG, num_inF_ATG, indicesTGG, indicesATG = return_inFrame(sequence)
    print(indicesTGG)
    print(num_inF_TGG)

[ 99 129 132]
3
[ 96 126 129]
3
[ 93 123 126]
3


In [137]:
for sequence in multiExon_sesRNAs:
    print(metric_gcContent(sequence))

0.6078431372549019
0.6078431372549019
0.6078431372549019


In [138]:
str(multiExon_sesRNAs[0])

'CTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGCAGCCAGGGAAGTGGGGGCCTGTGCGTTGAGGAGGCCAGATGGGAAAAGGTGGCCTCCGAGGAGCTCCGATGGTGGGTAAGTGGTGGAGTCCAGGTAGTTGAAGTAGTAGAGAGAGCCGCTGGCCGGCAGCCCCACAGCCTGGTT'

In [125]:
str(multiExon_sesRNAs[0].reverse_complement())

'AACCAGGCTGTGGGGCTGCCGGCCAGCGGCTCTCTCTACTACTTCAACTACCTGGACTCCACCACTTACCCACCATCGGAGCTCCTCGGAGGCCACCTTTTCCCATCTGGCCTCCTCAACGCACAGGCCCCCACTTCCCTGGCTGCTCACCCCAAGCTTTTTCTGCTGGAGAATGCCAAACTGGCCAGCCTGACTGCGGACAAG'

In [127]:
str(multiExon_sesRNAs[1].reverse_complement())

'ATCAACCAGGCTGTGGGGCTGCCGGCCAGCGGCTCTCTCTACTACTTCAACTACCTGGACTCCACCACTTACCCACCATCGGAGCTCCTCGGAGGCCACCTTTTCCCATCTGGCCTCCTCAACGCACAGGCCCCCACTTCCCTGGCTGCTCACCCCAAGCTTTTTCTGCTGGAGAATGCCAAACTGGCCAGCCTGACTGCGGAC'

In [126]:
str(multiExon_sesRNAs[2].reverse_complement())

'GTCATCAACCAGGCTGTGGGGCTGCCGGCCAGCGGCTCTCTCTACTACTTCAACTACCTGGACTCCACCACTTACCCACCATCGGAGCTCCTCGGAGGCCACCTTTTCCCATCTGGCCTCCTCAACGCACAGGCCCCCACTTCCCTGGCTGCTCACCCCAAGCTTTTTCTGCTGGAGAATGCCAAACTGGCCAGCCTGACTGCG'

In [24]:
for i in range(200, 300):
    if(i%3 == 0):
        print(i)

201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297


In [25]:
testSeq = sesRNAs[0]
testSeq

Seq('CTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGG...GTT')

In [26]:
str(testSeq)

'CTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGCAGCCAGGGAAGTGGGGGCCTGTGCGTTGAGGAGGCCAGATGGGAAAAGGTGGCCTCCGAGGAGCTCCGATGGTGGGTAAGTGGTGGAGTCCAGGTAGTTGAAGTAGTAGAGAGAGCCGCTGGCCGGCAGCCCCACAGCCTGGTT'

In [27]:
?split()

Object `split()` not found.


In [28]:
subsequence = str(testSeq)
codons = [subsequence for subsequence in re.split(r'(\w{3})', subsequence) if subsequence]

In [29]:
tempSeq = Seq('GTTCTCCTTCAGCACCTGCTCCAGCGGCGCATGCAAGCGCTCCTTATGGGGATAGGAAGCTGGGTGGGGGAACTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGCAGCCAGGGAAGTGGGGGCCTGTGCGTTGAGGAGGCCAGATGGGAAAAGGTGGCCTCCGAGGAGCTCCGATGG')

In [30]:
check_cORF(tempSeq)

True

In [31]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")

In [32]:
check_cORF(coding_dna)

False

## Outputing sesRNA

In [79]:
# Generating BioPython directory if does not exist 
pathlib.Path('Output/BioPython').mkdir(parents=True, exist_ok=True)

# Generate SeqRecord object for each sequence and append to list 
outputID = geneName + '_sesRNA'
outputDescription = "sesRNA for Fezf2"
outputSeqMulti = []
for i in multiExon_sesRNAs:
    outputSeqMulti.append(SeqRecord(i, id = outputID, description = outputDescription))
    
# Write output fasta files 
outputName = "Output/BioPython/Fezf2_sesRNA_V4.fasta"
with open(outputName, "w") as output_handle:
    SeqIO.write(outputSeqMulti, output_handle, "fasta")

In [80]:
len(multiExon_sesRNAs)

3

In [35]:
# Write output fasta files 
outputName = "Output/sesRNA_test.fasta"
with open(outputName, "w") as output_handle:
    SeqIO.write(outputSeqMulti[0], output_handle, "fasta")

In [36]:
str(outputSeqMulti[0].seq)

'CTTGTCCGCAGTCAGGCTGGCCAGTTTGGCATTCTCCAGCAGAAAAAGCTTGGGGTGAGCAGCCAGGGAAGTGGGGGCCTGTGCGTTGAGGAGGCCAGATGGGAAAAGGTGGCCTCCGAGGAGCTCCGATGGTGGGTAAGTGGTGGAGTCCAGGTAGTTGAAGTAGTAGAGAGAGCCGCTGGCCGGCAGCCCCACAGCCTGGTT'

# Secondary structure 

In [39]:
# Generating Temp 
pathlib.Path('Output/BioPython/Temp').mkdir(parents=True, exist_ok=True)

In [81]:
# Converting to RNA for calculating secondary structure 
sesRNAs_RNA = []
for i in range(len(multiExon_sesRNAs)):
    sesRNAs_RNA.append(sesRNAs[i].transcribe())

In [82]:
sesRNAs_RNA

[Seq('CUUGUCCGCAGUCAGGCUGGCCAGUUUGGCAUUCUCCAGCAGAAAAAGCUUGGG...GUU'),
 Seq('GUCCGCAGUCAGGCUGGCCAGUUUGGCAUUCUCCAGCAGAAAAAGCUUGGGGUG...GAU'),
 Seq('CGCAGUCAGGCUGGCCAGUUUGGCAUUCUCCAGCAGAAAAAGCUUGGGGUGAGC...GAC')]

In [84]:
str(sesRNAs_RNA[0])

'CUUGUCCGCAGUCAGGCUGGCCAGUUUGGCAUUCUCCAGCAGAAAAAGCUUGGGGUGAGCAGCCAGGGAAGUGGGGGCCUGUGCGUUGAGGAGGCCAGAUGGGAAAAGGUGGCCUCCGAGGAGCUCCGAUGGUGGGUAAGUGGUGGAGUCCAGGUAGUUGAAGUAGUAGAGAGAGCCGCUGGCCGGCAGCCCCACAGCCUGGUU'

In [68]:
# Writing sequences as seperate fasta files 
i = 1
for sesRNA in sesRNAs_RNA:
    outputName = geneName + '_' + str(i)
    outputDescription = "sesRNA #" + str(i)
    i += 1
    
    outputRecord = SeqRecord(sesRNA, id = outputName, description = outputDescription)
    outputFull = 'Output/BioPython/Temp/' + outputName + '.fasta'
    
    with open(outputFull, "w") as output_handle:
        SeqIO.write(outputRecord, output_handle, "fasta")

In [69]:
# Call RNAfold on each sequence of output 
rnaFold_prob = []
pathTemp = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp'
pathOutTemp = pathTemp + '/temp.out'

for entry in os.scandir(pathTemp):
    command = 'RNAfold -p -d2 --noLP < ' + entry.path + ' > ' + pathOutTemp    
    generateProb = subprocess.run(command, shell=True, stdout=subprocess.PIPE)

    # Moving to Temp directory to work on fasta files 
    currentWD = os.getcwd()
    os.chdir('Output/BioPython/Temp')
    
    # Running script for getting probabilities from RNAfold output file (added to ArchBin btw)
    readProb = subprocess.Popen("rnaFold_prob.sh", shell=True, stdout=subprocess.PIPE)
    returnedProb = readProb.stdout.read()
    # Waiting for last command to finish before storing value in temp.out file 
    readProb.wait()
    rnaFold_prob.append(float(returnedProb))
    
    # For checking which file currently working on (not in order for some reason) 
    print(entry.path)
    
    # Removing temp.out after finishing each run 
    os.system('rm -rf temp.out')
    # Return to initial working directory 
    os.chdir(currentWD)

/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp/Fezf2_2.fasta
/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp/Fezf2_1.fasta
/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Code/Output/BioPython/Temp/Fezf2_3.fasta


In [83]:
# Displaying ensemble frequency for secondary structures 
np.array(rnaFold_prob)*100

array([0.529741, 0.198473, 0.516524])

In [ ]:
from seqfold import dg, dg_cache, fold

In [ ]:
# just returns minimum free energy
dg("GGGAGGTCGTTACATCTGGGTAACACCGGTACTGATCCGGTGACCTCCC", temp = 37.0)  # -12.94

# Misc

In [ ]:
# For returning index of findings 
searchCodon = 'TAG'
[m.start() for m in re.finditer(searchCodon, str(seq))]

In [ ]:
testSeq = sesRNAs[0]
testSeq

In [ ]:
lastATG = [m.start() for m in re.finditer('ATG', str(testSeq))][-1]
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
lastTGG

In [ ]:
lastATG

In [ ]:
lastATG < lastTGG

In [ ]:
searchCodon = 'ATG'
[m.start() for m in re.finditer(searchCodon, str(testSeq))][-1]

In [ ]:
seq.count('TAG') < 4

In [ ]:
seq[0:100].count('TAG')

In [ ]:
stopCodons = ['TAG', 'TAA', 'TGA']
stopCodons 

In [ ]:
indiciesTGG

In [ ]:
indiciesStop

In [ ]:
length = 200 
center = length/2

In [ ]:
arrayStop = np.array(indiciesStop)
arrayIndicies = np.array(indiciesTGG) 
centralTGGs = arrayIndicies[abs(arrayIndicies - center) < 10]

In [ ]:
centralTGGs

In [ ]:
np.in1d(centralTGGs,arrayStop)

In [ ]:
# Check if array contains values that are within range of values in another array 
any((min(abs(arrayStop - i)) > 10) for i in centralTGGs)

In [ ]:
centralTGGs

In [ ]:
indiciesStop

In [ ]:
testStop = [90, 16, 174]

In [ ]:
(min(abs(arrayStop - centralTGGs[0])) > 10)

In [ ]:
min(abs(arrayStop - centralTGGs[0])) > 10

In [ ]:
centeralTGGs = offset.min()
centeralTGGs

In [ ]:
centralTGGs = np.all(offset == offset.min())
centralTGGs

In [ ]:
centeralTGGs = np.where(offset == offset.min())
centeralTGG

In [ ]:
offset = abs(arrayIndicies - center) 
centerTGG = indiciesTGG[np.argmin(offset)]

In [ ]:
any(abs(x - centerTGG) < 10 for x in indiciesStop)

In [ ]:
indiciesStop = []
for codons in stopCodons:
    indiciesStop.extend([m.start() for m in re.finditer(codons, str(testSeq))])

In [ ]:
len(indiciesStop)

In [ ]:
[m.start() for m in re.finditer('TGA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAA', str(testSeq))]

In [ ]:
[m.start() for m in re.finditer('TAG', str(testSeq))]

In [ ]:
testSeq.count(stopCodons)

In [ ]:
indiciesTGG

In [ ]:
len(indiciesTGG)

In [ ]:
testSeq = sesRNAs[0]

In [ ]:
lastTGG = [m.start() for m in re.finditer('TGG', str(testSeq))][-1]

In [ ]:
testSeq

In [ ]:
indiciesTGG = [m.start() for m in re.finditer('TGG', str(testSeq))]

In [ ]:
start = 0 
stop = 200

In [ ]:
middle = (start + stop) / 2

In [ ]:
abs(middle - indiciesTGG[0])

In [ ]:
type(indiciesTGG)

In [ ]:
indiciesTGG

In [ ]:
any(indiciesTGG) > 2

In [ ]:
length = 200 

In [ ]:
any(abs(x - (length/2)) < 20 for x in indiciesTGG)

In [ ]:
testList = [50, 60, 170, 200]

In [ ]:
any(abs(x - (length/2)) < 10 for x in testList)

In [ ]:
testSeq

In [ ]:
os.path.isdir('Output/BioPython')

In [ ]:
outputFileName = os.getcwd() + ''

In [ ]:
testSeq